<a href="https://colab.research.google.com/github/kranthireddy-29/Deep-Learning-Fundus-Image-Analysis-for-Early-Detection-of-Diabetic-Retinopathy-/blob/main/Diabetic_Retinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arbethi/diabetic-retinopathy-level-detection")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'diabetic-retinopathy-level-detection' dataset.
Path to dataset files: /kaggle/input/diabetic-retinopathy-level-detection


In [ ]:
# import os

# # Construct the full path to the zip file
# zip_file_path = os.path.join(path, 'diabetic-retinopathy-level-detection.zip')

# # Unzip the file
# # !unzip -q {zip_file_path} # This command failed as the zip file was not found.
# # It's likely that the dataset was automatically extracted by kagglehub.dataset_download.
# # You can inspect the contents of the 'path' directory to verify the downloaded content.

In [ ]:
import os

imageSize = [299, 299]

# Corrected paths based on inspection of the downloaded dataset
trainPath = os.path.join(path, 'preprocessed dataset', 'preprocessed dataset', 'training')
testPath = os.path.join(path, 'preprocessed dataset', 'preprocessed dataset', 'testing')

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                  target_size = (299, 299),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in xception.layers:
    layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(xception.output)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
prediction = Dense(5, activation='softmax')(x)

# create a model object
model = Model(inputs=xception.input, outputs=prediction)


In [ ]:
# view the structure of the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 21,885,485 (83.49 MB)

 Trainable params: 1,024,005 (3.91 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [ ]:
# fit the model
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set)//32,
    validation_steps=len(test_set)//32
)

NameError: name 'training_set' is not defined

In [ ]:
import os
import kagglehub # Added kagglehub import
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception

# Download latest version and define 'path'
path = kagglehub.dataset_download("arbethi/diabetic-retinopathy-level-detection")

# Define image size
imageSize = [299, 299]

# Corrected paths based on inspection of the downloaded dataset
trainPath = os.path.join(path, 'preprocessed dataset', 'preprocessed dataset', 'training')
testPath = os.path.join(path, 'preprocessed dataset', 'preprocessed dataset', 'testing')

# Initialize ImageDataGenerators
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Create training and test sets
training_set = train_datagen.flow_from_directory(trainPath,
                                                  target_size = (299, 299),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

# Build Xception model
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)

# Don't train existing weights
for layer in xception.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(xception.output)
prediction = Dense(5, activation='softmax')(x)

# Create a model object
model = Model(inputs=xception.input, outputs=prediction)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Fit the model
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set)//32,
    validation_steps=len(test_set)//32
)

# Save the model after training
model.save('Updated-Xception-diabetic-retinopathy.h5')

100%|██████████| 9.66G/9.66G [08:27<00:00, 20.4MB/s]

Extracting files...


Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 402s 189s/step - accuracy: 0.3763 - loss: 5.8145 - val_accuracy: 0.4973 - val_loss: 12.5396
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 379s 182s/step - accuracy: 0.5742 - loss: 7.5708 - val_accuracy: 0.5531 - val_loss: 6.4390
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 381s 184s/step - accuracy: 0.5729 - loss: 5.7822 - val_accuracy: 0.5613 - val_loss: 4.0342
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 368s 177s/step - accuracy: 0.5911 - loss: 5.3870 - val_accuracy: 0.5926 - val_loss: 4.1170
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 426s 206s/step - accuracy: 0.5951 - loss: 4.5803 - val_accuracy: 0.6757 - val_loss: 3.2409
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 427s 206s/step - accuracy: 0.6393 - loss: 4.3741 - val_accuracy: 0.6267 - val_loss: 3.2610
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 426s 206s/step - accuracy: 0.7370 - loss: 3.1689 - val_accuracy: 0.6703 - val_loss: 2.8995
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 386s 185s/step - accuracy: 0.6458 - loss: 3.9773 - val_accuracy: 0.6553 - 

In [ ]:
model.save('Updated_Xception_diabetic_retinopathy.keras')

In [ ]:
model = load_model("Updated-Xception-diabetic-retinopathy.h5", compile=False)